In [1]:
#Importing Packages
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor
import datetime as dt
import pandas as pd
from sklearn import preprocessing, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
%matplotlib inline
GroupedShoes = pd.read_csv('GroupedShoes.csv')

In [2]:
GroupedShoes

,Unnamed: 0,Sneaker Name,Shoe Size,Retail Price,Resell Price,Release Date,Order Date,Red,Black,White,...,Neo,Orange,Tan/Brown,Pink,Blue,Colorful,release_season,release_year,order_season,order_year
0,0,adidas-yeezy-boost-350-low-v2-beluga,4.0,220,746.500000,2016-09-24,2017-09-07,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Fall 2016,2016,Fall 2016,2017
1,1,adidas-yeezy-boost-350-low-v2-beluga,6.0,220,897.500000,2016-09-24,2017-09-05,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Fall 2016,2016,Fall 2016,2017
2,2,adidas-yeezy-boost-350-low-v2-beluga,6.5,220,900.000000,2016-09-24,2017-09-25,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Fall 2016,2016,Fall 2016,2017
3,3,adidas-yeezy-boost-350-low-v2-beluga,7.0,220,795.000000,2016-09-24,2017-09-15,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Fall 2016,2016,Fall 2016,2017
4,4,adidas-yeezy-boost-350-low-v2-beluga,7.5,220,832.000000,2016-09-24,2017-09-06,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,Fall 2016,2016,Fall 2016,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,831,nike-zoom-fly-off-white-pink,12.0,170,272.490385,2018-11-28,2018-11-27,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,Fall 2018,2018,Fall 2018,2018
832,832,nike-zoom-fly-off-white-pink,12.5,170,297.142857,2018-11-28,2018-12-02,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,Fall 2018,2018,Winter 2018,2018
833,833,nike-zoom-fly-off-white-pink,13.0,170,272.000000,2018-11-28,2018-11-28,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,Fall 2018,2018,Fall 2018,2018
834,834,nike-zoom-fly-off-white-pink,14.0,170,307.400000,2018-11-28,2018-11-28,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,Fall 2018,2018,Fall 2018,2018


In [3]:
X = GroupedShoes.drop(['Resell Price',"release_year","order_year",'Release Date','Order Date'], axis=1)
y = GroupedShoes['Resell Price']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [4]:
X_train.shape

(668, 16)

In [5]:
# Converting categorical data to numerical
from sklearn.preprocessing import OneHotEncoder

object_cols = ['Sneaker Name','release_season','order_season']
# Apply one-hot encoder to each column with categorical data
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_valid = pd.DataFrame(OH_encoder.transform(X_valid[object_cols]))

# One-hot encoding removed index; put it back
OH_cols_train.index = X_train.index
OH_cols_valid.index = X_valid.index

# # Adding the column names after one hot encoding
OH_cols_train.columns = OH_encoder.get_feature_names(object_cols)
OH_cols_valid.columns = OH_encoder.get_feature_names(object_cols)

# Remove categorical columns (will replace with one-hot encoding)
num_X_train = X_train.drop(object_cols, axis=1)
num_X_valid = X_valid.drop(object_cols, axis=1)

# Add one-hot encoded columns to numerical features
X_train = pd.concat([num_X_train, OH_cols_train], axis=1)
X_valid = pd.concat([num_X_valid, OH_cols_valid], axis=1)

In [6]:
#MODEL TESTING
#1. Linear Regression
#2. OLS Regression
#3. RandomForestRegressor
#4. DecisionTreeRegressor
#5. XGBoost

In [7]:
#Linear Regression 

In [8]:
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [9]:
# Looking at y-int
print(lm.intercept_)

12004.246734226266


In [10]:
# Storing predictions and running evaluation metrics
predictions = lm.predict(X_valid)
from sklearn import metrics
print("MAE:", metrics.mean_absolute_error(y_valid, predictions))
print('MSE:', metrics.mean_squared_error(y_valid, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_valid, predictions)))

MAE: 84.86360405190416
MSE: 34217.212607226254
RMSE: 184.97895179513333


In [11]:
#OLS Regression
def build_model(X,y):
    X = sm.add_constant(X) #Adding the constant
    model = sm.OLS(y, X)
    results = model.fit() # fitting the model
    print(results.summary()) # model summary
    return X
    
def checkVIF(X):
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)

In [12]:
X_train_new = build_model(X_valid,y_valid)

                            OLS Regression Results                            
Dep. Variable:           Resell Price   R-squared:                       0.896
Model:                            OLS   Adj. R-squared:                  0.853
Method:                 Least Squares   F-statistic:                     20.77
Date:                Fri, 24 Mar 2023   Prob (F-statistic):           1.88e-39
Time:                        17:16:05   Log-Likelihood:                -1009.3
No. Observations:                 168   AIC:                             2119.
Df Residuals:                     118   BIC:                             2275.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
                                                                     coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------

In [13]:
# SETING UP MODEL PIPELINE

In [14]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor

In [15]:
# Setting up pipelines

from sklearn.pipeline import Pipeline
# Decision Tree Regression Pipeline
pipeline_dtr=Pipeline([('dtr', DecisionTreeRegressor(random_state=27))])

# Random Forest Pipeline
pipeline_randomforest=Pipeline([('rf_regressor',RandomForestRegressor(random_state=27))])

# XGBost Pipeline
pipeline_xgb=Pipeline([('xgb_regressor',xgb.XGBRegressor(objective="reg:linear", random_state=27))])

In [16]:
# Creating a list of the pipelines to loop through them
pipelines = [pipeline_dtr, pipeline_xgb, pipeline_randomforest]

best_accuracy=0.0
best_regressor=0
best_pipeline=""

# Dictionary of pipelines and regression types for ease of reference
pipe_dict = {0: 'DTR', 1: 'XGBoost', 2: 'RandomForest'}

# Fit the pipelines
for pipe in pipelines:
	pipe.fit(X_train, y_train)

from sklearn.metrics import accuracy_score
# Checking the accuracy of each model
for i,model in enumerate(pipelines):
    print("{} Test Accuracy: {}".format(pipe_dict[i],model.score(X_valid,y_valid)))

# Finding the best model
for i,model in enumerate(pipelines):
    if model.score(X_valid,y_valid)>best_accuracy:
        best_accuracy=model.score(X_valid,y_valid)
        best_pipeline=model
        best_regressor=i
print('Model with best accuracy: {}'.format(pipe_dict[best_regressor]))

[17:16:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[17:16:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.2.0/src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
DTR Test Accuracy: 0.8017599225481001
XGBoost Test Accuracy: 0.8841805941563221
RandomForest Test Accuracy: 0.846915703251563
Model with best accuracy: XGBoost


In [17]:
#Hyperparameters
#Random Forest Regresson

In [18]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

In [19]:
# Using Randomized Search CV to find the best parameters

# Number of trees in random forest
n_estimators = [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 25, 50, 75, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

pprint(random_grid)

{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 5, 10],
 'min_samples_split': [2, 5, 10, 15, 25, 50, 75, 100],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [20]:

rf = RandomForestRegressor(random_state=27)
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=27, n_jobs = -1)

# Fit the random search model

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:   13.9s remaining:   13.9s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   18.4s finished


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=27),
                   n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 5, 10],
                                        'min_samples_split': [2, 5, 10, 15, 25,
                                                              50, 75, 100],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=27, verbose=2)

In [21]:
# Evaluation of Random Search
def evaluate(model, X_valid, y_valid):
    predictions = model.predict(X_valid)
    errors = np.sqrt(mean_squared_error(y_valid, predictions))
    print('Model Performance')
    print('MSE of: ', errors)
    
    return errors

In [22]:
from sklearn.metrics import mean_squared_error
base_model = rf
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_valid, y_valid)


best_random = rf_random.best_estimator_
best_random.fit(X_train , y_train)

random_accuracy = evaluate(best_random, X_valid, y_valid)

print('\n')
print('Base Model Error: ', base_accuracy)
print('\n')
print('Improved Model Error: ', random_accuracy)
print('Improvement of {:0.2f}%.'.format((random_accuracy - base_accuracy) / base_accuracy))

print('\n')
print('RF_Randomized_Search_CV is complete.')
print('\n')

Model Performance
MSE of:  119.41852821882215
Model Performance
MSE of:  119.59141760442951


Base Model Error:  119.41852821882215


Improved Model Error:  119.59141760442951
Improvement of 0.00%.


RF_Randomized_Search_CV is complete.




In [23]:
print('The best model is',rf_random.best_estimator_)

The best model is RandomForestRegressor(max_depth=90, min_samples_split=5, n_estimators=2000,
                      random_state=27)


In [24]:
print( rf_random.best_score_)

0.8892066944136557


In [25]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(best_random, X_valid, y_valid,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)
print("Average MAE score (across experiments):\n")
print(scores.mean())

MAE scores:
 [ 73.55084572  94.85283721 137.62887786  59.55171599 153.17930747]
Average MAE score (across experiments):

103.75271684918043


In [26]:
# Saving model to disk
import pickle
pickle.dump(best_random, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))